In [1]:
from config import *
# from metric import compute_metrics_for_regression
from dataloader import TextDataset
import pandas as pd
from sklearn.model_selection import train_test_split

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [2]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    logits = logits[0]
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    
    return {"mse": mse, "mae": mae, "r2": r2}

In [ ]:
df = pd.read_csv('../../data/frame2.csv')[['description_clear', 'salary_from_rate_and_gross_log']]

In [ ]:
import numpy as np
df['salary_from_rate_and_gross_log'] = np.exp(df['salary_from_rate_and_gross_log'])

In [ ]:
df.head(2)

In [ ]:
train_dataset, valid_dataset = train_test_split(df, test_size=0.3, random_state=0)
valid_dataset, test_dataset = train_test_split(valid_dataset, test_size=0.5, random_state=0)

In [ ]:
test_dataset

In [ ]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

ds = DatasetDict(
    {
        "train": Dataset.from_dict({'label': train_dataset['salary_from_rate_and_gross_log'], 'text': train_dataset['description_clear']}),
        "valid": Dataset.from_dict({'label': valid_dataset['salary_from_rate_and_gross_log'], 'text': valid_dataset['description_clear']}),
        "test": Dataset.from_dict({'label': test_dataset['salary_from_rate_and_gross_log'], 'text': test_dataset['description_clear']}),
    }
)

In [ ]:
def preprocess_function(examples):
    label = examples['label']
    examples = tokenizer(
                            examples['text'],
                            add_special_tokens=True,
                            max_length=MAX_LENGTH,
                            return_token_type_ids=False,
                            truncation=True,
                            padding='max_length',
                            return_attention_mask=False,
                            return_tensors='pt',
    )
    examples["label"] = label
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=["label", "text"])

In [3]:
from datasets import load_from_disk
# ds.save_to_disk("./dataset")
ds = load_from_disk("./dataset")

c:\Users\maksk\Anaconda3\lib\site-packages\datasets\dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [4]:
ds['train'][0]

{'label': 45199.999999999956,
 'input_ids': [[2,
   105,
   815,
   15976,
   1439,
   324,
   25313,
   16687,
   656,
   23939,
   751,
   117,
   2389,
   322,
   3098,
   27095,
   626,
   312,
   19387,
   17947,
   6253,
   7332,
   3995,
   2182,
   23881,
   705,
   778,
   1172,
   4144,
   13158,
   25415,
   25696,
   1229,
   320,
   25067,
   14889,
   20046,
   324,
   25313,
   1229,
   23498,
   13305,
   778,
   314,
   10138,
   18765,
   22638,
   324,
   25313,
   17947,
   15164,
   23423,
   342,
   2698,
   721,
   11521,
   28377,
   2253,
   15480,
   776,
   28194,
   2225,
   12384,
   6455,
   11521,
   3789,
   2454,
   21496,
   28618,
   11521,
   2389,
   2179,
   15687,
   1241,
   320,
   4154,
   644,
   9611,
   25415,
   26150,
   7274,
   22079,
   1276,
   3995,
   4452,
   2371,
   317,
   3881,
   1023,
   987,
   7405,
   21010,
   866,
   5677,
   20534,
   26629,
   21431,
   18603,
   22793,
   24495,
   22860,
   23498,
   18754,
   11521,


In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="regression_model",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=50,
    num_train_epochs=20,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    weight_decay=0.01,
    # prediction_loss_only=True,
    # use_legacy_prediction_loop=True
)

In [6]:
import torch

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        outputs = model(inputs['input_ids'])
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [7]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["valid"],
    # compute_metrics=compute_metrics_for_regression,
)

trainer.train()

c:\Users\maksk\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 109936
  Num Epochs = 20
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 22000
  2%|▏         | 502/22000 [00:47<35:46, 10.02it/s] 

{'loss': 5631750176.768, 'learning_rate': 1.9545454545454546e-05, 'epoch': 0.45}


  5%|▍         | 1001/22000 [01:33<32:08, 10.89it/s]

{'loss': 5656811143.168, 'learning_rate': 1.9090909090909094e-05, 'epoch': 0.91}


  5%|▌         | 1100/22000 [01:43<31:29, 11.06it/s]***** Running Evaluation *****
  Num examples = 23558
  Batch size = 50
                                                    
  5%|▌         | 1100/22000 [01:49<31:29, 11.06it/s]Saving model checkpoint to regression_model\checkpoint-1100
Configuration saved in regression_model\checkpoint-1100\config.json
Model weights saved in regression_model\checkpoint-1100\pytorch_model.bin


...

                                                     
100%|██████████| 22000/22000 [36:16<00:00, 11.09it/s]Saving model checkpoint to regression_model\checkpoint-22000
Configuration saved in regression_model\checkpoint-22000\config.json
Model weights saved in regression_model\checkpoint-22000\pytorch_model.bin


{'eval_loss': 5626724352.0, 'eval_runtime': 6.5985, 'eval_samples_per_second': 3570.221, 'eval_steps_per_second': 71.532, 'epoch': 20.0}


Deleting older checkpoint [regression_model\checkpoint-19800] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from regression_model\checkpoint-22000 (score: 5626724352.0).
100%|██████████| 22000/22000 [36:16<00:00, 10.11it/s]

{'train_runtime': 2176.764, 'train_samples_per_second': 1010.087, 'train_steps_per_second': 10.107, 'train_loss': 5644194700.567273, 'epoch': 20.0}


TrainOutput(global_step=22000, training_loss=5644194700.567273, metrics={'train_runtime': 2176.764, 'train_samples_per_second': 1010.087, 'train_steps_per_second': 10.107, 'train_loss': 5644194700.567273, 'epoch': 20.0})

In [8]:
from transformers import AutoModelForSequenceClassification
model_new = AutoModelForSequenceClassification.from_pretrained('regression_model/checkpoint-20900/', local_files_only=True)

loading configuration file regression_model/checkpoint-20900/config.json
Model config BertConfig {
  "_name_or_path": "regression_model/checkpoint-20900/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29564
}

loading weights file regression_model/checkpoint-20900/pytorch_model.bin
All mod

In [9]:
z = torch.Tensor(ds['test'][0]['input_ids'])
z = z.to(dtype=torch.int32, device='cuda')

In [12]:
ds['test'][0]['label']

30000.000000000007

In [10]:
trainer.model(z)[0]

tensor([[74.1521]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
trainer.model(z)[0][:, 0]

tensor([74.1521], device='cuda:0', grad_fn=<SelectBackward0>)

In [ ]:
outputs[0][:, 0]

In [ ]:
ds['test'][0]['label']

In [ ]:
model_new.cuda()
model_new(z)

In [ ]:
ds['test'].num_rows

In [ ]:
from tqdm import tqdm
import numpy as np

model_new.cuda()

def prediction(dataset):
    df = np.zeros((ds['test'].num_rows, 2))
    for i, data in enumerate(tqdm(dataset)):
        label = data['label']
        data = data['input_ids']
        data = torch.Tensor(data)
        data = data.to(dtype=torch.int32, device='cuda')
        label_hat = model_new(data)[0].detach().cpu().numpy()[0][0]

        df[i, :] = [label_hat, label]
        if i % 1000 == 0:
            print(i)
    return df

array = prediction(ds['test'])

In [ ]:
array

In [ ]:
[(x, x**2) for x in range(10)]